In [59]:
import data
import models
import cache
import stanity
%matplotlib inline

In [60]:
sns.set(context='talk')

In [70]:
import logging
cache_logger = logging.getLogger('cache')
cache_logger.setLevel(logging.INFO)

In [62]:
by = 'cell_type'
sample_n = 500

## load data

Load data as we did in previous notebooks, so we can compare predictive performance for particular observations

In [26]:
sample_df = cache.cached(models.prep_sample_df, sample_n=sample_n)

INFO:cache:prep_sample_df: Loading result from cache


In [27]:
stan_data = models.prep_stan_data(sample_df, by=by)

## model3 -- using poisson distribution

Load results from model3 from cache

In [76]:
model3 = models.get_model_file(model_name='model3')
fit3 = models.cached_stan_fit(file=model3,
                              fit_cachefile='model3.model_code_73307426795.stanfit.chains_81232192355.data_35729880489.iter_31194724242.seed_57902074806.pkl')

INFO:cache:Step 1: Get compiled model code, possibly from cache
INFO:cache:StanModel: Loading result from cache
INFO:cache:Step 2: Get posterior draws from model, possibly from cache
INFO:cache:sampling: Loading result from cache
INFO:cache:Fit cachefile: model3.model_code_73307426795.stanfit.chains_81232192355.data_35729880489.iter_31194724242.seed_57902074806.pkl


In [79]:
models.print_stan_summary(fit3, pars='lp__')

              mean   se_mean         sd          2.5%           50%         97.5%      Rhat
lp__  5.281721e+07  1.534292  30.609025  5.281715e+07  5.281721e+07  5.281727e+07  1.014739


## model4 -- using negative binomial dist

Load results from fitting model4 from cache

In [71]:
model4 = models.get_model_file(model_name='model4')
fit4 = models.cached_stan_fit(file=model4,
                              fit_cachefile='model4.model_code_35447780597.stanfit.chains_81232192355.data_35729880489.iter_31194724242.seed_57902074806.pkl'
                             )

INFO:cache:Step 1: Get compiled model code, possibly from cache
INFO:cache:StanModel: Loading result from cache
INFO:cache:Step 2: Get posterior draws from model, possibly from cache
INFO:cache:sampling: Loading result from cache
INFO:cache:Fit cachefile: model4.model_code_35447780597.stanfit.chains_81232192355.data_35729880489.iter_31194724242.seed_57902074806.pkl


In [78]:
models.print_stan_summary(fit4, pars='lp__')

              mean   se_mean         sd          2.5%           50%         97.5%      Rhat
lp__  5.383965e+07  2.126517  36.954924  5.383958e+07  5.383966e+07  5.383972e+07  1.002845


## Compare models using LOO

Summarize Leave-One-Out (LOO) predictive performance for model3 & model4, using pareto-smoothed importance sampling (PSIS) method to approximate true CV performance.

In [ ]:
loo3 = cache.cached(stanity.psisloo, log_likelihood=fit3.extract('log_lik')['log_lik'],
                   cache_filename='model3_psisloo.by_{}.sample_n_{}.pkl'.format(by, sample_n),
                   force=False)

In [ ]:
loo3.print_summary()

In [ ]:
loo4 = cache.cached(stanity.psisloo, log_likelihood=fit4.extract('log_lik')['log_lik'],
                   cache_filename='model4_psisloo.by_{}.sample_n_{}.pkl'.format(by, sample_n),
                   force=False)

In [ ]:
loo4.print_summary()

Compare fit using model3 & model4

In [ ]:
stanity.loo_compare(loo3, loo4)

A larger value suggests that model4 is a better fit than model3. How much better can be evaluated intuitively by comparing the magnitude of the difference to the SE of the difference.